<a href="https://colab.research.google.com/github/lbralexis/OrangeGIt/blob/main/Orange25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install chromium, its driver, and selenium
!apt update
!apt install libu2f-udev libvulkan1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip
!unzip -j chromedriver-linux64.zip chromedriver-linux64/chromedriver -d /usr/local/bin/
!pip install selenium chromedriver_autoinstaller
!pip install pytz
!pip install selenium webdriver-manager

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,770 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restrict

In [ ]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

# 📌 Configuration de Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 📌 Initialisation WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

# 📌 URL de la page listant les téléphones
main_page_url = "https://boutique.orange.fr/mobile/mobiles-et-smartphones?brand=apple,samsung,google,xiaomi,honor&status=neuf&offer=forfait-200go-5g&page=2"

# 📌 Fonction pour récupérer toutes les URLs des produits
def get_product_links(url):
    print(f"📌 Scraping des produits sur : {url}")
    driver.get(url)
    time.sleep(3)

    try:
        section = driver.find_element(By.CLASS_NAME, "models-wrapper")
        links = section.find_elements(By.TAG_NAME, "a")

        # ✅ Filtrer uniquement les liens contenant "/mobile/details/"
        filtered_links = [link.get_attribute("href") for link in links if link.get_attribute("href") and "/mobile/details/" in link.get_attribute("href")]
        print(f"🔗 {len(filtered_links)} URLs de produits trouvées.")
        return filtered_links
    except Exception as e:
        print(f"❌ Erreur lors de la récupération des liens : {e}")
        return []

# 📌 Fonction pour extraire la capacité d'une URL
def extract_capacity(url):
    match = re.search(r"-(\d+go|\d+to)", url, re.IGNORECASE)
    return match.group(1) if match else None

# 📌 Fonction pour générer une URL avec une nouvelle capacité
def replace_capacity_in_url(url, new_capacity):
    return re.sub(r"-(\d+go|\d+to)", f"-{new_capacity}", url, 1)

# 📌 Fonction pour scraper les données générales (H1, Capacités, Bonus)
def scrape_general_data(url):
    print(f"🔗 Ouverture de l'URL : {url}")
    driver.get(url)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))

    # ✅ Récupération du titre H1
    try:
        h1_text = driver.find_element(By.TAG_NAME, "h1").text.strip()
        print(f"✅ H1 récupéré : {h1_text}")
    except Exception:
        h1_text = ""

    # ✅ Récupération des capacités disponibles
    capacities = []
    try:
        fieldsets = driver.find_elements(By.TAG_NAME, "fieldset")
        for fieldset in fieldsets:
            legend = fieldset.find_element(By.TAG_NAME, "legend").text.strip()
            if "Capacités disponibles" in legend:
                labels = fieldset.find_elements(By.TAG_NAME, "label")
                capacities = [label.text.strip().lower().replace(" ", "") for label in labels]
                print(f"📦 Capacités trouvées : {capacities}")
                break
    except Exception:
        capacities = []

    # ✅ Récupération du bonus reprise
    try:
        accordion_button = driver.find_element(By.ID, "refunds-button")
        driver.execute_script("arguments[0].scrollIntoView();", accordion_button)

        if accordion_button.get_attribute("aria-expanded") == "false":
            driver.execute_script("arguments[0].click();", accordion_button)
            time.sleep(2)

        bonus_text = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li.row:nth-of-type(2) .col-12.col-md-4.fw-bold.amount span"))).text.strip()
        print(f"🎁 Bonus reprise récupéré : {bonus_text}")
    except Exception:
        bonus_text = "0"
        print("❌ Aucun bonus trouvé, valeur mise à 0")

    return h1_text, capacities, bonus_text

# 📌 Fonction pour scraper le prix
def scrape_price(url):
    print(f"🔄 Récupération du prix pour : {url}")
    driver.get(url)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ob1-price-amount")))

    try:
        price_text = driver.find_element(By.CLASS_NAME, "ob1-price-amount").text.strip()
        print(f"💰 Prix récupéré : {price_text}")
        return price_text
    except Exception:
        print("❌ Erreur lors de la récupération du prix")
        return ""

# 📌 Fonction pour scraper la date d'expiration de la remise
def scrape_discount_info(url):
    driver.get(url)
    time.sleep(2)

    try:
        expiration_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".d-lg-flex .mt-1 span:nth-of-type(2)")))
        expiration_date = expiration_element.text.strip()
        print(f"📅 Date d'expiration trouvée : {expiration_date}")
        return expiration_date
    except Exception:
        print("❌ Aucune date d'expiration trouvée.")
        return ""

# 📌 Liste pour stocker toutes les données
all_product_data = []

# 📌 Récupération de toutes les URLs de produits
product_links = get_product_links(main_page_url)

# 📌 Scraping de chaque produit
for base_url in product_links:
    base_capacity = extract_capacity(base_url)

    # ✅ Scraper les données générales (H1, Capacités, Bonus)
    h1_title, capacities, base_bonus = scrape_general_data(base_url)

    # ✅ Scraper la date de remise
    expiration_date = scrape_discount_info(base_url)

    # ✅ Scraper le prix de la version de base
    base_price = scrape_price(base_url)
    all_product_data.append((base_url, base_capacity, h1_title, base_price, expiration_date, base_bonus))

    # ✅ Scraper les autres capacités uniquement pour le prix
    for capacity in capacities:
        if capacity != base_capacity:
            modified_url = replace_capacity_in_url(base_url, capacity)
            price = scrape_price(modified_url)
            all_product_data.append((modified_url, capacity, h1_title, price, expiration_date, base_bonus))

driver.quit()

# 📌 Création du DataFrame
df = pd.DataFrame(all_product_data, columns=["URL", "Capacité", "Titre H1", "Prix", "Date", "Bonus Reprise"])

# 📌 Nettoyage des valeurs manquantes
df.fillna("N/A", inplace=True)

# 📌 Tri par "Titre H1" puis par "Capacité"
df.sort_values(by=["Titre H1", "Capacité"], inplace=True)

# 📌 Affichage propre du DataFrame
print("📊 Aperçu des données :")
print(df.head())

# 📌 Génération d'un nom de fichier avec horodatage
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"capacities_prices_bonus_{timestamp}.csv"

# 📌 Sauvegarde en CSV
df.to_csv(filename, index=False)
print(f"✅ Données sauvegardées dans : {filename}")


📌 Scraping des produits sur : https://boutique.orange.fr/mobile/mobiles-et-smartphones?brand=apple,samsung,google,xiaomi,honor&status=neuf&offer=forfait-200go-5g&page=2
🔗 59 URLs de produits trouvées.
🔗 Ouverture de l'URL : https://boutique.orange.fr/mobile/details/xiaomi-15-ultra-gris-512go?offer=forfait-200go-5g
✅ H1 récupéré : Xiaomi 15 Ultra
📦 Capacités trouvées : ['512go']
🎁 Bonus reprise récupéré : 300 €
❌ Aucune date d'expiration trouvée.
🔄 Récupération du prix pour : https://boutique.orange.fr/mobile/details/xiaomi-15-ultra-gris-512go?offer=forfait-200go-5g
💰 Prix récupéré : 849 €
🔗 Ouverture de l'URL : https://boutique.orange.fr/mobile/details/honor-magic7-pro-5g-noir-512go?offer=forfait-200go-5g
✅ H1 récupéré : HONOR Magic7 Pro 5G+
📦 Capacités trouvées : ['512go']
🎁 Bonus reprise récupéré : 150 €
❌ Aucune date d'expiration trouvée.
🔄 Récupération du prix pour : https://boutique.orange.fr/mobile/details/honor-magic7-pro-5g-noir-512go?offer=forfait-200go-5g
💰 Prix récupéré : 69